## Notebook for tuning a CNN model

**Install HYPERAS libraries first**

In [ ]:
!pip install hyperas
!pip install hyperopt

**Load libraries and modules**

In [ ]:
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe

import numpy as np

from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.backend import clear_session

from tqdm import tqdm


**Mount google drive to read numpy array files**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**HYPERAS data function**

In [ ]:
from numpy import load

def data():
  # load array from numpy array files
  X_train = load('/content/drive/My Drive/images_for_training/X_train.npy')
  y_train = load('/content/drive/My Drive/images_for_training/y_train.npy')
  X_test = load('/content/drive/My Drive/images_for_training/X_test.npy')
  y_test = load('/content/drive/My Drive/images_for_training/y_test.npy')
  
  return X_train, y_train, X_test, y_test


**HYPERAS model function**

In [ ]:
def model(x_train, y_train, x_test, y_test):  

    clear_session()

    model = Sequential()
    model.add(
        Conv2D(
            {{choice([26, 52])}},
            kernel_size = {{choice([(3, 3),(5,5)])}},
            padding = 'valid',
            activation = 'relu',
            input_shape = (200, 200, 1)
        )
    )
    model.add(
        Conv2D(
            {{choice([26, 52])}}, 
            kernel_size = {{choice([(3, 3),(5,5)])}},
            padding = "valid",
            activation='relu'
        )
    )
    model.add(
        MaxPooling2D(
            pool_size = {{choice([(2, 2),(4,4)])}}
        )
    )
    ### Should try removing these ###
    #################################
    model.add(
        Conv2D(
            {{choice([20, 40])}},
            kernel_size = {{choice([(3, 3),(5,5)])}},
            padding = "valid",
            activation='relu'
        )
    )
    model.add(
        MaxPooling2D(
            pool_size = {{choice([(2, 2),(4,4)])}}
        )
    )
    #################################
    model.add(
        Dropout(
            {{uniform(0, 1)}}
        )
    )
    model.add(Flatten())
    model.add(
        Dense(
            {{choice(
                [256, 512])}}
        )
    )
    model.add(
        Activation(
            {{choice(['relu', 'tanh'])}}
        )
    )
    model.add(
        Dropout(
            {{uniform(0, 1)}}
        )
    )
    model.add(Dense(26, activation='softmax'))

    model.summary()

    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    callback = EarlyStopping(monitor='val_loss', patience=3)

    result = model.fit(X_train, y_train, epochs={{choice([5,10,20])}}, verbose=2, validation_split=0.1, callbacks=[callback])
    
    json_string = model.to_json()

    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(result.history['val_accuracy']) 
    print('Best validation acc of epoch:', validation_acc)

    return {'loss': -validation_acc, 'status': STATUS_OK, 'model':(json_string)}

In [ ]:
# Look at how much RAM memory the varibles are using.
# import sys
# # These are the usual ipython objects, including this one you are creating
# ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
# # Get a sorted list of the objects and their sizes
# sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

In [ ]:
# Testing how many parameters are being generated by the model.
# model = Sequential()
# model.add(
#         Conv2D(
#             64,
#             kernel_size = (3,3),
#             padding = 'valid',
#             activation = 'relu',
#             input_shape = (200, 200, 1)
#         )
#     )
# model.add(
#         Conv2D(
#             64, 
#             kernel_size = (3, 3),
#             padding = "valid",
#             activation='relu'
#         )
#     )

# model.add(
#         MaxPooling2D(
#             pool_size = (2, 2)
#         )
#     )
# model.add(
#         Conv2D(
#             20, 
#             kernel_size = (3, 3),
#             padding = "valid",
#             activation='relu'
#         )
#     )
# model.add(
#         MaxPooling2D(
#             pool_size = (4, 4)
#         )
#     )
# model.add(
#         Dropout(
#             .25
#         )
#     )
# model.add(Flatten())
# model.add(
#         Dense(
#             256
#         )
#     )
# model.add(
#         Activation(
#             'tanh'
#         )
#     )
# model.add(
#         Dropout(
#             .25
#         )
#     )
# model.add(Dense(26, activation='softmax'))

# Print the summary of the model.
# model.summary()



In [ ]:
# Compile the model with testing data.
# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# callback = EarlyStopping(monitor='val_loss', patience=3)

# Fit the model with test data
# result = model.fit(X_test, y_test, epochs=50, verbose=2, validation_split=0.1, callbacks=[callback])


Workaround to allow the notebook name to be recognized by HYPERAS

In [ ]:
# See: https://stackoverflow.com/questions/49920031/get-the-path-of-the-notebook-on-google-colab
# Install the PyDrive wrapper & import libraries.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Copy/download the file
fid = drive.ListFile({'q':"title='HyperasModelTuning.ipynb'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('HyperasModelTuning.ipynb')

**Run the HYPERAS optimization process**

In [ ]:
# Get best parameters to train model
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5, # should try to use 3
                                      trials=Trials(),
                                      notebook_name='HyperasModelTuning')

X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from tensorflow.keras.layers import Conv2D
except:
    pass

try:
    from tensorflow.keras.callbacks import EarlyStopping
except:
    pass

try:
    from tensorflow.keras.layers import MaxPooling2D
except:
    pass

try:
    from tensorflow.keras.layers import Flatten
except:
    pass

try:
    from tensorflow.keras.preprocessing import image
except:
    pass

try:


**Save the tuned model.**

In [ ]:
model.save('/content/drive/My Drive/model/tuned_model_1.h5', save_format='h5')
